In [5]:
# -------------------------------------
# --- Go to correct starting folder ---
# -------------------------------------
# (when running jupyter lab in the browser, the notebook starts with CWD = folder where it is located, which breaks imports, ...)
import os
import pathlib

while not ((cwd := pathlib.Path(os.getcwd())) / "requirements.txt").exists():
    os.chdir(cwd.parent)  # go 1 folder up

In [6]:
# other imports
from collections import defaultdict

import numpy as np
from matplotlib import pyplot as plt
from PIL import Image

from core.data import SearchResult
from core.search import semantic_search, textual_search
from notebooks.colors import CLR_BLACK, CLR_BLUE, CLR_DARK_GREEN, CLR_GREEN, CLR_GREY
from notebooks.helpers import enable_metadata, get_data_folder, get_figures_folder, get_images_folder

# 1. Introduction

This notebook will run the 7 semantic search queries that we used in threshold analysis and will automatically split the results into 3 folders (TP, FP, TN) and resize all results.

In [7]:
# queries we will test
queries = ["dog", "coffee", "food", "sheep", "rabbit", "snow", "bike"]
ground_truth: dict[str, list[str]] = dict()  # ground truth results (file names) for each query

all_files = [file.name for file in get_images_folder().glob("*.jpg")]
print(f"ALL FILES: {len(all_files)}")

for query in queries:
    ground_truth_path = get_data_folder() / "semantic_search" / "ground_truth" / query
    ground_truth[query] = [str(file.name) for file in ground_truth_path.glob("*.jpg")]

print("GROUND TRUTH")
for query, results in ground_truth.items():
    print(f"   {query.ljust(10)} -> {len(results)} results")

ALL FILES: 150
GROUND TRUTH
   dog        -> 2 results
   coffee     -> 9 results
   food       -> 36 results
   sheep      -> 9 results
   rabbit     -> 2 results
   snow       -> 15 results
   bike       -> 12 results


In [8]:
def resize_img_to_max_dim(img: Image.Image, max_dim: int) -> Image.Image:
    """
    Resize an image to fit within a square of max_dim x max_dim pixels.
    """
    width, height = img.size
    if width > height:
        new_width = max_dim
        new_height = int(height * (max_dim / width))
    else:
        new_height = max_dim
        new_width = int(width * (max_dim / height))

    return img.resize((new_width, new_height), Image.LANCZOS)


def copy_image(src_filename: str, dst_file_index: str, query: str, result_type: str):
    """
    Copy an image to the correct folder based on the query and result type (TP, FP, TN).
    We copy one version that has max dimensions 1024 and one that has max dimensions 256.
    """

    print(f"   processing image {dst_file_index} - {src_filename} in {query}/{result_type}")

    # --- load image ---
    img_path = get_images_folder() / src_filename
    img = Image.open(img_path)
    img_small = resize_img_to_max_dim(img, max_dim=200)
    img_large = resize_img_to_max_dim(img, max_dim=1000)

    # --- save again ---
    dst_folder = get_data_folder() / "semantic_search" / "results" / query / result_type
    dst_folder.mkdir(parents=True, exist_ok=True)

    img_small.save(dst_folder / f"img_{dst_file_index:0>3}_small.jpg", quality=90)
    img_large.save(dst_folder / f"img_{dst_file_index:0>3}_large.jpg", quality=80)


# go query by query
for query in queries:
    print(f"QUERY: {query}")

    # --- perform query -----------------------------------
    results = semantic_search(
        directory=get_images_folder(),
        query=query,
        min_score=0.49,  # optimal value found in threshold analysis
    )
    result_filenames = [result.filename for result in results]

    # --- split in TP, FP, TN -----------------------------
    tp = [filename for filename in result_filenames if filename in ground_truth[query]]
    fp = [filename for filename in result_filenames if filename not in ground_truth[query]]
    fn = [filename for filename in ground_truth[query] if filename not in result_filenames]

    # --- copy files --------------------------------------
    for i, tp_filename in enumerate(tp, start=1):
        copy_image(tp_filename, i, query, "tp")

    for i, fp_filename in enumerate(fp, start=1):
        copy_image(fp_filename, i, query, "fp")

    for i, fn_filename in enumerate(fn, start=1):
        copy_image(fn_filename, i, query, "fn")

QUERY: dog
   processing image 1 - 2025.03.27-18.22.22-MOB-0001.jpg in dog/tp
   processing image 2 - 2025.03.27-18.22.23-MOB-0001.jpg in dog/tp
QUERY: coffee
   processing image 1 - 2025.05.30-12.18.29-MOB-0001.jpg in coffee/tp
   processing image 2 - 2025.05.30-12.18.57-MOB-0001.jpg in coffee/tp
   processing image 3 - 2025.05.30-08.21.35-MOB-0001.jpg in coffee/tp
   processing image 4 - 2025.06.03-11.13.11-MOB-0001.jpg in coffee/tp
   processing image 5 - 2025.05.28-12.58.48-MOB-0001.jpg in coffee/tp
   processing image 6 - 2025.05.28-12.58.37-MOB-0001.jpg in coffee/tp
   processing image 7 - 2025.05.27-07.39.56-MOB-0001.jpg in coffee/tp
   processing image 8 - 2025.06.02-07.56.55-MOB-0001.jpg in coffee/tp
   processing image 1 - 2025.05.27-07.39.52-MOB-0001.jpg in coffee/fn
QUERY: food
   processing image 1 - 2025.05.24-17.33.49-MOB-0001.jpg in food/tp
   processing image 2 - 2025.06.01-18.25.26-MOB-0001.jpg in food/tp
   processing image 3 - 2025.05.26-18.25.44-MOB-0001.jpg in foo